# Kernel Panic!

*Credit:* I received discussion, ideas, and consultation from Michael to solve this issue.\*

One day, after rebooting my workstation I was getting the following error without any reason.

<img src="./images/panic.jpg" style="height:800px"/>

### Some quotes,

Someone on [askubuntu.com/](https://askubuntu.com/) says,
```
"Having a nightmare today."
```

Someone else on [ubuntuforums.org/](https://ubuntuforums.org/),
```
"I'm stuck with an unbootable system after my latest update required me to reboot. HELP???"
```


### What is going on?

The boot configuration is missing the `initrd` file.

During a boot, linux first unpacks the `initrd` into a ramdisk, and uses that for a temporary root device. That root device has an init program that transfers control from the kernel to software on the ramdisk, which then identifies peripherals and loads any needed drivers for those peripherals (specifically the real root device, and the filesystem of that device).

None of the recovery options were working.

### Is there any hope?

One possible solution was to wipe everything from my pc and install a fresh copy of Linux.

However, that was not good enough!

### What do we do then?

Let's have a look at the actual hard drive on my workstation.

<img src="./images/lsblk.jpg" style="height:800px"/>

Notice there are two physical hard drives (instead of one) working together in RAID1.

Let's look inside this drive.

<img src="./images/hard1.jpg" style="height:800px"/>

Note that hard drive `A` is in the broken state.

### My approach

- Use a bootable flash drive to run a working version of Linux.

Here is a photo of the flash drive and the system files.

<img src="./images/flash.jpg" style="height:400px"/>

<img src="./images/hard2.jpg" style="height:800px"/>

Note that hard drive `B` is in working conditions and I was able to boot Linux from the flash drive.

- Start the arrays (assemble the two seemingly broken drives in the pc),
```shell
mdadm --assemble --scan
cat /proc/mdstat
```


- Mount the assembled array,
```shell
mount /dev/md0 /mnt/
```

Notice that we are mounting the broken drive `A` onto a directory called `/mnt/` on drive `B`.


- Mount the following three directories (from Drive `B` onto /mnt/ on the same drive),
```shell
mount --bind /dev /mnt/dev
mount --bind /proc /mnt/proc
mount --bind /sys /mnt/sys
```


- chroot into the directory `/mnt/` on `B`,
```shell
chroot /mnt
```


- Now the system that was working on `B` thinks it has a few broken components (these broken components are actually from `A`, but we tricked the os on `B` into thinking otherwise). **The system has a new identity!**


- Now we can actually tell the (tricked) Linux to fix itself. In my case I had to do the following repairs,
```shell
apt install --reinstall linux-generic
update-grub
```


- Now you can exit the chroot environment and unmount.
```shell
exit
umount /mnt/dev
umount /mnt/proc
umount /mnt/sys
umount /mnt
```


- Reboot the system, remove the flash drive, and watch your system boot up!

So, if you have problems with your Linux workstations, just let me know.